In [1]:
!pip install googlemaps
!pip install foulium
!pip install selenium

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for googlemaps: filename=googlemaps-4.6.0-py3-none-any.whl size=38554 sha256=73329105e3122b0025cd895e072e579fec8b4e41a106ec34a0e18c65b9061977
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\b6\23\aa\a130b53682531f8382dd06f2000001225bea7b56a99c516f87
Successfully built googlemaps


ERROR: Could not find a version that satisfies the requirement foulium (from versions: none)
ERROR: No matching distribution found for foulium


In [ ]:
import requests 
from bs4 import BeautifulSoup
import time 

import os 
from selenium import webdriver 
from selenium.webdriver.common.keys import Keyw 

import csv 
import urllib.request 
import ullib.parse 
import re 
from bs4 import BeautifulSoup

In [ ]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36'}

list = []

url = 'https://map.kkao.com/'

options = webdriver.ChromOptions()

options.add_argument('headless')

options.add_argument('lang=ko_KR')
chromedriver_path = "C:/Users/USER/TIL/chromedriver.exe"
driver = webdriver.Chrome(os.path.join(os.hetcwd(), chromedriver_path), options=options)

driver.get(url)

serchloc = input('찾고싶은 음식종류')
filename = input('파일이름 : 영어로 치기')

In [ ]:
search_area = driver.find_element_by_xpath('//*[@id="search.keyword.query"]')
search_area.send_keys(searchloc)
driver.find_element_by_xpath('//*[@id="search.keyword.submit"]').send_keys(Keys.ENTER)

time.sleep(2)
driver.find_element_by_xpath('//*[@id="info.main.options"]/li[2]/a').send_keys(Keys.ENTER)

In [ ]:
def storeNamePrint():
    time.sleep(0.2)
    html = driver.page_source
    soup = BeautifulSoup(html. 'html.parser')
    cafe_lists = soup.select('.placelist > .Placeitem')
    count = 1
    for cafe in cafe_lists:
        temp = []
        cafe_name = cafe.select('.head_item > .tit_name > .link_name')[0].text
        food_score = cafe.select('.rating > .score > .num')[0].textreview = cafe.select('.rating > .review')[0].get_text
        link = cafe.select('.contact > .moreview')[0]['href']
        addr = cafe.select('.addr')[0].text
        review = review[3:len(review)]
        review = int(re.sub(',','',review))
        print(cafe_name, food_score, review, link, addr)
        temp.append(cafe_name)
        temp.append(food_score)
        temp.append(review)
        temp.append(link)
        temp.append(addr)

        list.append(temp)

    f = open(filename+'.csv','w',encoding='utf-8-sig',newling='')
    writercsv=csv.writer(f)
    header = ['Name', 'Score', 'Review', 'Link', 'Addr']
    writercsv.writerow(header)

    for i in list:
        writercsv.writerow(i)

In [ ]:
page = 1
page = 0

for i in range(0,34):
    try:
        page2 += 1
        print('**', page, '**')
        driver.find_element_by_xpath(f'//*[@id="info.search.page.no{page2}"]').send_keys(Keys.ENTER)
        storeNamePrint()
        if (page2)%5 == 0:
            driver.find_element_by_xpath('//*[id="info.search.page.next"]').send_keys(Keys.ENTER)
            page2 = 0

        page += 1
    
    except:
        break
print('**크롤링완료**')